# Import ploonetide and other libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ploonetide import TidalSimulation
from ploonetide.utils import colorline
from ploonetide.utils.functions import mean2axis, find_moon_fate
from ploonetide.utils.constants import GYEAR, KYEAR, DAY, MSUN, AU

## Create now the TidalSimulation object

In [ ]:
simulation = TidalSimulation(
    system='planet-moon',
    planet_orbperiod=20,
    moon_eccentricty=0.0,
    moon_semimaxis=10,
    planet_size_evolution=False,
    planet_internal_evolution=False,
    planet_core_dissipation=False,
)

### Choose the total integration time and time-step

In [ ]:
integration_time = 1 * simulation.stellar_lifespan
timestep = 100 * KYEAR

### Choose the ingrator and run the simulation

In [ ]:
simulation.set_integration_method('rk4')
simulation.run(integration_time, timestep)

### Get the times and solutions

In [ ]:
times, solutions = simulation.history
nms = solutions[:, 0]
ops = solutions[:, 1]
nps = solutions[:, 2]
Tms = solutions[:, 3]
Ems = solutions[:, 4]
if simulation.moon_eccentricty != 0.0:
    ems = solutions[:, 5]